# Bobrick & Martire: Physical Warp Drives

This notebook explores the Bobrick & Martire warp drive classification.

**Reference**: Bobrick, A. & Martire, G. (2021). Class. Quantum Grav. 38, 105009.

## Key Innovation

Bobrick & Martire showed that:
1. **Subluminal ($v < c$)** warp drives can satisfy energy conditions
2. **Positive-energy configurations** exist for slow-enough velocities
3. A general "warp shell" framework classifies all warp drives

This was the first demonstration that warp drives don't inherently require exotic matter - only superluminal ones do.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from warpbubblesim.metrics import AlcubierreMetric, BobrickMartireMetric
from warpbubblesim.gr import compute_energy_density, check_energy_conditions
from warpbubblesim.gr.geodesics import integrate_geodesic
from warpbubblesim.viz.fields2d import plot_energy_density

%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams['font.size'] = 12

## 1. The Warp Shell Framework

Bobrick & Martire define a general warp drive as a "warp shell" with three regions:

- **Interior**: Passenger region (approximately flat, moving with velocity $v$)
- **Shell**: Where the metric deviates from flat (exotic or normal matter)
- **Exterior**: Asymptotically flat (stationary far away)

In [ ]:
# Create Bobrick-Martire metric
bm = BobrickMartireMetric(
    v0=0.3,           # Subluminal!
    R_inner=1.0,      # Inner shell radius
    R_outer=2.0,      # Outer shell radius
    shell_amplitude=0.1,
    positive_energy=True
)

info = bm.info()
print(f"Metric: {info['name']}")
print(f"Citation: {info['citation']}")
print(f"Parameters: {info['parameters']}")
print(f"\nEnergy condition type: {bm.energy_condition_type()}")

## 2. Subluminal vs Superluminal

The critical distinction: **velocity determines whether exotic matter is needed**

In [ ]:
# Compare energy conditions at different velocities
velocities = [0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 1.5]

print("Energy Conditions vs Velocity (positive_energy=True configuration):")
print("="*70)
print(f"{'Velocity':<10} {'WEC':<12} {'NEC':<12} {'Min ρ':<15}")
print("-"*70)

for v in velocities:
    try:
        metric = BobrickMartireMetric(v0=v, positive_energy=True)
        metric_func = metric.get_metric_func()
        
        # Check in the shell region
        coords = np.array([0, 1.5, 0.3, 0])  # Shell region
        conditions = check_energy_conditions(metric_func, coords, n_samples=20)
        
        wec = "OK" if conditions['WEC'][0] else "VIOLATED"
        nec = "OK" if conditions['NEC'][0] else "VIOLATED"
        
        rho = compute_energy_density(metric_func, coords)
        
        label = "subluminal" if v < 1 else "luminal" if v == 1 else "superluminal"
        print(f"{v:<10.1f} {wec:<12} {nec:<12} {rho:<15.2e} ({label})")
    except Exception as e:
        print(f"{v:<10.1f} Error: {e}")

## 3. Positive Energy Configuration

For subluminal velocities, the positive-energy configuration can satisfy all energy conditions:

In [ ]:
# Detailed energy condition check for subluminal positive-energy drive
bm = BobrickMartireMetric(v0=0.2, positive_energy=True)
metric_func = bm.get_metric_func()

# Check at multiple points
test_points = [
    ('Interior', [0, 0, 0, 0]),
    ('Shell inner', [0, 1.2, 0.2, 0]),
    ('Shell middle', [0, 1.5, 0.3, 0]),
    ('Shell outer', [0, 1.8, 0.2, 0]),
    ('Exterior', [0, 3.0, 0, 0]),
]

print(f"Energy conditions for v={bm.v0}c (subluminal, positive-energy):")
print("="*70)

for name, coords in test_points:
    coords = np.array(coords, dtype=float)
    conditions = check_energy_conditions(metric_func, coords, n_samples=30)
    
    print(f"\n{name} {coords[1:]}:")
    for cond_name, (satisfied, value) in conditions.items():
        status = "OK" if satisfied else "VIOLATED"
        print(f"  {cond_name}: {status:8s} ({value:+.4e})")

## 4. Energy Density Comparison

In [ ]:
# Compare Alcubierre (requires exotic) vs B&M (positive energy)
alc = AlcubierreMetric(v0=0.3)  # Same velocity
bm = BobrickMartireMetric(v0=0.3, positive_energy=True)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

plot_energy_density(alc, x_range=(-3, 3), y_range=(-3, 3), 
                    nx=80, ny=80, ax=axes[0])
axes[0].set_title(f'Alcubierre v={alc.v0}c\n(requires exotic matter)')

plot_energy_density(bm, x_range=(-3, 3), y_range=(-3, 3), 
                    nx=80, ny=80, ax=axes[1])
axes[1].set_title(f'Bobrick-Martire v={bm.v0}c\n(positive energy possible)')

plt.tight_layout()
plt.show()

## 5. Shell Structure

In [ ]:
bm = BobrickMartireMetric(v0=0.3, R_inner=1.0, R_outer=2.0, positive_energy=True)
metric_func = bm.get_metric_func()

# Energy density along radius
r = np.linspace(0, 4, 200)
rho = [compute_energy_density(metric_func, np.array([0, ri, 0.1, 0])) for ri in r]

plt.figure(figsize=(10, 6))
plt.plot(r, rho, 'b-', linewidth=2)
plt.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
plt.axvline(x=bm.R_inner, color='green', linestyle='--', alpha=0.5, label=f'R_inner={bm.R_inner}')
plt.axvline(x=bm.R_outer, color='red', linestyle='--', alpha=0.5, label=f'R_outer={bm.R_outer}')

# Shade regions
plt.axvspan(0, bm.R_inner, alpha=0.1, color='blue', label='Interior')
plt.axvspan(bm.R_inner, bm.R_outer, alpha=0.1, color='yellow', label='Shell')
plt.axvspan(bm.R_outer, 4, alpha=0.1, color='gray', label='Exterior')

plt.xlabel('r (distance from center)')
plt.ylabel(r'Energy density $\rho$')
plt.title('Bobrick-Martire Warp Shell Structure')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 6. Velocity Threshold

There exists a critical velocity below which positive-energy configurations are possible:

In [ ]:
# Find transition velocity
velocities = np.linspace(0.05, 1.0, 20)
min_energies = []

for v in velocities:
    try:
        bm = BobrickMartireMetric(v0=v, positive_energy=True)
        metric_func = bm.get_metric_func()
        
        # Sample energy density in shell
        samples = []
        for x in np.linspace(1.0, 2.0, 10):
            for y in np.linspace(0, 0.5, 5):
                rho = compute_energy_density(metric_func, np.array([0, x, y, 0]))
                samples.append(rho)
        min_energies.append(min(samples))
    except:
        min_energies.append(np.nan)

plt.figure(figsize=(10, 6))
plt.plot(velocities, min_energies, 'b-o', linewidth=2)
plt.axhline(y=0, color='red', linestyle='--', label='Zero energy density')
plt.xlabel('Velocity (units of c)')
plt.ylabel('Minimum energy density in shell')
plt.title('Energy Density vs Velocity\n(Negative values = exotic matter required)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

# Find threshold
for i, (v, e) in enumerate(zip(velocities, min_energies)):
    if e < 0:
        print(f"\nTransition to negative energy around v ~ {velocities[max(0,i-1)]:.2f}c")
        break

## 7. Geodesics

In [ ]:
bm = BobrickMartireMetric(v0=0.3, positive_energy=True)
metric_func = bm.get_metric_func()

# Integrate geodesics
results = []
for x0 in np.linspace(-4, 4, 9):
    initial_coords = np.array([0.0, x0, 0.0, 0.0])
    initial_velocity = np.array([1.0, 0.0, 0.0, 0.0])
    
    result = integrate_geodesic(
        metric_func, initial_coords, initial_velocity,
        lambda_span=(0, 20), max_step=0.1
    )
    results.append(result)

plt.figure(figsize=(12, 8))

for result in results:
    coords = result['coords']
    plt.plot(coords[:, 1], coords[:, 0], alpha=0.7)

# Bubble trajectory (subluminal)
t_range = np.linspace(0, 20, 100)
x_bubble = [bm.bubble_center(t) for t in t_range]
plt.plot(x_bubble, t_range, 'k--', linewidth=2, label=f'Bubble (v={bm.v0}c)')

# Light cone reference
plt.plot([0, 20], [0, 20], 'r:', alpha=0.5, label='Light (v=c)')

plt.xlabel('x')
plt.ylabel('t')
plt.title('Geodesics in Bobrick-Martire Spacetime (Subluminal)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 8. Energy Requirements

In [ ]:
bm = BobrickMartireMetric(v0=0.3, positive_energy=True)

energy = bm.energy_estimate()

print("Energy Requirements for Bobrick-Martire Subluminal Warp:")
print("="*50)
print(f"Velocity: {energy['velocity']}c")
print(f"Positive energy: {energy['positive_energy']}")
print(f"Shell thickness: {bm.R_outer - bm.R_inner}")
print(f"\nKey advantage: No exotic matter required!")
print(f"Trade-off: Limited to subluminal velocities")

## 9. Comparison Table

In [ ]:
print("Warp Drive Classification (Bobrick & Martire Framework):")
print("="*70)
print(f"{'Type':<20} {'Velocity':<15} {'Exotic Matter':<20} {'Feasibility':<15}")
print("-"*70)

configs = [
    ('Alcubierre', 'Any', 'Required', 'Speculative'),
    ('Natário', 'Any', 'Required', 'Speculative'),
    ('B&M Superluminal', 'v > c', 'Required', 'Speculative'),
    ('B&M Subluminal', 'v < c', 'Not required', 'Less speculative'),
]

for name, vel, exotic, feasibility in configs:
    print(f"{name:<20} {vel:<15} {exotic:<20} {feasibility:<15}")

## 10. Physical Implications

The Bobrick & Martire result is significant because:

1. **Subluminal warp drives are less exotic** - no fundamental violations of known physics
2. **Engineering constraints shift** - from "find exotic matter" to "build extreme mass configurations"
3. **Still requires enormous energy** - positive doesn't mean easy
4. **Speed of light remains limit** - but relativistic travel without propellant is still valuable

In [ ]:
# Practical implications
print("Travel Times at Different Subluminal Velocities:")
print("="*50)

distances = [
    ('Moon', 1.3),       # Light-seconds
    ('Mars (avg)', 720),  # Light-seconds
    ('Alpha Centauri', 4.37 * 365.25 * 24 * 3600),  # Light-years to seconds
]

for name, dist_ls in distances:
    print(f"\n{name} ({dist_ls:.1e} light-seconds):")
    for v in [0.1, 0.3, 0.5, 0.9]:
        time_s = dist_ls / v
        if time_s < 3600:
            print(f"  v={v}c: {time_s:.0f} seconds")
        elif time_s < 86400:
            print(f"  v={v}c: {time_s/3600:.1f} hours")
        elif time_s < 365.25*86400:
            print(f"  v={v}c: {time_s/86400:.1f} days")
        else:
            print(f"  v={v}c: {time_s/(365.25*86400):.1f} years")

## Summary

The Bobrick & Martire warp drive:

1. **First demonstration** of positive-energy warp drives (subluminal only)
2. **General framework** for classifying all warp drive spacetimes
3. **Subluminal = possible** (with extreme but normal matter)
4. **Superluminal = exotic matter required** (confirms previous results)
5. **Practical applications** still require enormous energies
6. **Theoretical significance** - warp drives aren't inherently impossible

This represents a major step forward in understanding what types of warp drives might be physically realizable.